# CS202: Compiler Construction

## In-class Exercises, Week of 03/21/2022

----

# Part 1: While Loops

## Question 1

Compile the following program to $\mathcal{C}_{if}$:

```
i = 10
sum = 0
while i > 0:
    i = i - 1
    sum = sum + i
print(sum)
```

Create test_label for the test, body_label for the body statements test_label has an if statement that goes to body_label if the test passes body_label goes back to test_label shen it's finished.
```
start:
	i = 10
	sum = 0
	goto test_label
test_label:
	if i > 0: goto body_label
body_label:
	i = i - 1
	sum = sum + i
	goto test_label
cont_label:
	print(sum)
	goto conclusion
```

## Question 2

Compile the program above into pseudo-x86 assembly.

```
start:
	movq $10, i
	movq $0, sum
	jmp test_label
test_label:
	cmpq $0, i
	jgt body_label
	jmp cont_label
body_label:
	subq $1, i
	addq i, sum
	jmp test_label
cont_label:
	movq sum, %rdi
	call print_int
	jmp conclusion
```

## Question 3

Describe the major changes to the compiler, up to *select instructions*.

- Typechecker:
    - Add a case for while
        - Test is a boolean
        - Body statements are well-typed
- RCO
    - Add a case for while
    - While is not an operator, so its subexpression don't need to be atomic
    - Call rco_exp on the test, with atomic=False
        - Generate a Begin node for the test, that includes the new bindings from calling rco_exp on the test
        - Begin(stmts, exp) is an AST node that runs stmts then evaluates to exp
    - Call rco_stmts on the body statements
- Explicate Control
    - From above:
        1. Create test_label for the test, body_label for the body statements
        2. test_label has an if statement that goes to body_label if the test passes
        3. body_label goes back to test_label shen it's finished
    - Implementation:
        In explicate_stmts, with a current_label and cont_label:
        For the case While(Begin([], test), body_stmts):
        1. Create labels (test_label. body_label, new_cont_label)
        2. Compile test into new_test w/ explicate_exp
        3. Add if statement to test_label:
            cif.If(new_test, cif.Goto(body_label), cif.Goto(new_cont_label))
        4. Compile body_stmts w/ current_label = body_label, cont_label = test_label (so that when the loop body finishes, it jumps back to the test)
        5. Set current_label = new_cont_label
- Select Instructions       **No changes**
- Liveness Analysis
	- Become a dataflow analysis
- Build Interference Graph  **No changes**
- Allocate Registers        **No changes**
- Patch Instructions        **No changes**
- Print x86                 **No changes**

# Part 2: Dataflow Analysis

## Question 4

Perform liveness analysis on the pseudo-x86 program from Question 2.

- Computing live-before for test_label requires knowing live-before for body_label
- Computing live-before for body_label requires knowing live-before for test_label
- Thus, we're in an infinite loop

```
start:
	movq $10, i
	movq $0, sum    {
	jmp test_label  {}
test_label:
	cmpq $0, i
	jgt body_label  {sum}
	jmp cont_label  {}
body_label:
	subq $1, i
	addq i, sum     {
	jmp test_label  {}
cont_label:         {sum}
	movq sum, %rdi  {}
	call print_int  {}
	jmp conclusion  {}
```

## Question 5

Describe the idea of dataflow analysis on cyclic control-flow graphs.

(Section 5.2)

Dataflow analysis allows us to analyze cyclic CFGs.

Process:
1. Start with an under-approximation of the answer
2. Iteratively improve the answer until you reach a fixpoint

Under-approximation = potentially vail answer for some executions of the program
Over-approximation = valid answer for All executions of the program
Fixpoint = run the analysis again => get the same answer

For liveness analysis:
1. Starting under-approximation = all blocks have empty live-before sets
2. Repeat the analysis until all blocks live-before sets stay the same

## Question 6

Use the dataflow-based approach to perform liveness analysis on the pseudo-x86 program above.

```
start:              {}
	movq $10, i     {i}
	movq $0, sum    {i, sum}
	jmp test_label  {}
test_label:         {i, sum}
	cmpq $0, i      {i, sum}
	jgt body_label  {sum}
	jmp cont_label  {}
body_label:         {i, sum}
	subq $1, i      {i, sum}
	addq i, sum     {i, sum}
	jmp test_label  {}
cont_label:         {sum}
	movq sum, %rdi  {}
	call print_int  {}
	jmp conclusion  {}
```

## Question 7

How do we know the dataflow analysis will stop (i.e. not loop forever)?

- Every program has a finite number of variables
- Our analysis performs unions on sets of variables
- In the worst case, every variable will be live at every program location
- In this case, the live-before sets will eventually grow to include all variables
- If I run my analysis again, there's nothing left to add to those sets(they already have all the variables in them)
- So I have reached a fixpoint

## Question 8

What changes are required for the rest of the compiler?

No changes are required.